In [1]:
import pandas as pd
import os
def data_dictionary():
    path_train="dataset/"
    list_train=os.listdir(path_train)
    train_dictionary={"image_path":[],"target":[]}
    k=0
    for i in list_train:
        
        path_disease_train=path_train+i
        image_list_train=os.listdir(path_disease_train)
        for j in image_list_train:
            img_path_train=path_disease_train+"/"+j   
            train_dictionary["image_path"].append(img_path_train)
            train_dictionary['target'].append(k)     
        k+=1
    train_df=pd.DataFrame(train_dictionary)
    return  train_df

In [3]:
data=data_dictionary()

In [14]:

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [36]:
import cv2
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog

In [38]:
import cv2
images=[]
label=[]
hog_features=[]

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
for i in data['image_path']:
    
    img=cv2.imread(i,0)
    img=cv2.resize(img,(100,100))
    hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))               
    hog_features.append(hog_feature)
    images.append(img)
                   

In [40]:
hog_features=np.array(hog_features)

In [44]:
lable=data['target']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(hog_features, lable, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)



In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)

In [57]:
model=Sequential([
                  Dense(1024,activation='relu'),
                   Dense(512,activation='relu'),
                  Dense(256,activation='relu'),
                  Dense(23,activation='softmax'),

])

In [60]:
import keras
num_classes = 23
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [61]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [63]:
history=model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=32,callbacks=[mcp_save,reduce_lr],epochs=70)

Epoch 1/70
587/587 [==============================] - 13s 22ms/step - loss: 2.0820 - accuracy: 0.3574 - val_loss: 2.6411 - val_accuracy: 0.2463 - lr: 0.0010
Epoch 2/70
587/587 [==============================] - 13s 22ms/step - loss: 2.0599 - accuracy: 0.3670 - val_loss: 2.6408 - val_accuracy: 0.2496 - lr: 0.0010
Epoch 3/70
587/587 [==============================] - 13s 22ms/step - loss: 2.0286 - accuracy: 0.3763 - val_loss: 2.5807 - val_accuracy: 0.2540 - lr: 0.0010
Epoch 4/70
587/587 [==============================] - 13s 22ms/step - loss: 2.0135 - accuracy: 0.3767 - val_loss: 2.5853 - val_accuracy: 0.2573 - lr: 0.0010
Epoch 5/70
587/587 [==============================] - 13s 22ms/step - loss: 1.9914 - accuracy: 0.3836 - val_loss: 2.5829 - val_accuracy: 0.2563 - lr: 0.0010
Epoch 6/70
587/587 [==============================] - 13s 22ms/step - loss: 1.9757 - accuracy: 0.3862 - val_loss: 2.6607 - val_accuracy: 0.2511 - lr: 0.0010
Epoch 7/70
587/587 [==============================] - 13s 

In [65]:
y_test = keras.utils.to_categorical(y_test, num_classes)

In [66]:
_, acc = model.evaluate(X_test, y_test)

73/73 [==============================] - 0s 5ms/step - loss: 3.0820 - accuracy: 0.3795


In [67]:
y_pred=model.predict(X_test)

73/73 [==============================] - 0s 5ms/step


In [73]:
y_pred=np.argmax(y_pred, axis=1)
y_pred

array([18,  0,  4, ..., 11,  1, 20])

In [74]:
y_test=np.argmax(y_test, axis=1)

In [75]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


In [76]:
confusion_matrix_result=confusion_matrix(y_test,y_pred)

In [77]:
confusion_matrix_result

array([[39,  3,  0,  3,  1,  2,  7,  1,  3,  4,  4,  3,  1,  1,  1,  2,
         2,  2,  3,  0,  4,  3,  2],
       [ 0, 23,  6,  1,  3,  4,  5,  3,  4,  2,  5,  3,  3,  1,  4,  0,
         5,  2,  5,  3,  1,  4,  1],
       [ 2,  4, 60,  7,  3,  1,  1,  0,  2,  4,  0,  5,  7,  2,  4,  1,
         3,  0,  0,  3,  0,  1,  1],
       [ 5,  2,  1, 46,  2,  1,  3,  2,  3,  4, 15,  2,  3,  0,  0,  2,
         0,  4,  0,  5,  2,  2,  8],
       [ 2,  5,  2,  1, 45,  0,  4,  2,  5,  1,  5,  1,  5,  4,  7,  1,
         2,  1,  2,  1,  2,  2,  2],
       [ 0,  0,  0,  0,  2, 46,  3,  0,  5,  2,  1,  2,  3,  3,  2,  7,
         6,  2,  1,  5,  3,  3,  5],
       [ 3,  0,  5,  1,  1,  5, 44,  0,  1,  1,  1,  2,  5,  3,  4,  1,
         1,  4,  5,  4,  2,  4,  1],
       [ 2,  0,  1,  2,  9,  4,  3, 36,  1,  0,  1,  1,  5,  1,  8,  1,
         2,  3,  1,  1,  1,  3,  1],
       [ 1,  1,  1,  1,  3,  6,  5,  0, 35,  5,  4,  3,  4,  1,  1,  4,
        10,  1,  1,  6,  4,  1,  8],
       [ 4,  1,  1,

In [ ]:
classification_report